## Librerias
Es necesario instalar selenium, requests no puede acceder al cuerpo del html de la página, y se necesita time para que la página no ponga el mensaje de "confirma que no eres un robot"

In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver                  # hay que instalar este paquete
import numpy as np
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
def getAndParseURL(url):
    driver = webdriver.Firefox()#options=opts)    # si no usan firefox cambian esta linea por el nombre de su navegador
    driver.get(url)

    page_source = driver.page_source
    soup = bs(page_source,features="html.parser")
    time.sleep(5)
    driver.close()
    return soup

In [4]:
def encontrar_ano(string):
    anos = [f'20{i}{j}' for j in range(10) for i in range(3)]
    anos2 = [f'199{j}' for j in range(10)]
    anos += anos2
    for ano in anos:
        if ano in string:
            return ano
    return np.nan

In [23]:
def get_marca(string):
    marcas = ['kia', 'hyundai', 'honda', 'chery', 'mg', 'mazda', 'suzuki']
    for marca in marcas:
        if marca in string.lower():
            return marca
    return np.nan

def get_modelo(string):
    modelos = ['accent', 'rio-5', 'wr-v', 'cr-v', '2', '3', 'tiggo-2', 'baleno', 'xv', 'swift']
    for modelo in modelos:
        if modelo in string.lower():
            return modelo
    return np.nan

In [6]:
# para obtener todas las páginas
def get_pag_sgte(soup):
    next_page = soup.find('a', {'data-webm-clickvalue':'sv-bpg-next'})
    #print(next_page)
    try:
        url = next_page.get('href')
        return url
    except:
        print('ultima página')
        return False

In [7]:
def get_info_autos(soup):
    datos = []
    publicaciones = soup.find_all('div', class_='card-body')
    for pub in publicaciones:
        try:
            titulo = pub.find('a', class_='js-encode-search')
            marca = get_marca(titulo.text.strip('\n').replace(' ', ''))
            modelo = get_modelo(titulo.text.strip('\n').replace(' ', ''))
            ano = encontrar_ano(titulo.text.strip('\n'))
        except:
            marca = np.nan
            modelo = np.nan
            ano = np.nan
        try:
            kilometraje = pub.find('li', {'class':'key-details__value', 'data-type':'Odometer'}).text.strip('\n')
        except:
            kilometraje = np.nan
        try:
            transmision = pub.find('li', {'class':'key-details__value', 'data-type':'Transmission'}).text.strip('\n')
        except: 
            transmision = np.nan
        #precio
        try:
            precio = pub.find('div', class_='price').a.text
            precio = precio.replace('$', '').replace(',', '').replace('CLP', '').replace(' ', '')
            #precio = precio.replace(',', '')
        except:
            precio = np.nan
        datos.append((marca, modelo, ano, kilometraje, transmision, precio))

    return datos

In [20]:
def contruir_df_auto(marca, modelo):
    df = pd.DataFrame(columns=['Marca', 'Modelo', 'Año', 'Kilometraje', 'Transmision', 'Precio'])
    i = 1
    url_og = f'https://www.chileautos.cl/vehiculos/autos-veh%C3%ADculo/{marca}/{modelo}/'
    seguir = True

    while seguir:   # mientras haya una sgte página
        if i == 1:
            soup = getAndParseURL(url_og) 
        else:
            soup = getAndParseURL(url)
        datos_pag = get_info_autos(soup)
        for publicacion in datos_pag:
            marca_auto, modelo_auto, ano, kilometraje, transmision, precio = publicacion
            df = df.append({'Marca':marca_auto, 'Modelo':modelo_auto, 'Año':ano, 'Kilometraje':kilometraje, 'Transmision':transmision, 'Precio': precio}, \
                ignore_index=True)
            #serie = pd.Series({'Marca':[marca_auto], 'Modelo':[modelo_auto], 'Año':[ano], 'Kilometraje':[kilometraje], 'Transmision':[transmision], 'Precio': [precio]})
            #df = pd.concat([df, serie], axis=0)
        i += len(datos_pag) + 1
        if i >= 150:
            return df
        url = get_pag_sgte(soup)
        if url != False:
            try:
                url = 'https://www.chileautos.cl' + url
            except:
                seguir = False
        else:
            seguir = False
        time.sleep(10)
    
    return df

In [111]:
df_wrv = contruir_df_auto('honda', 'wr-v')

<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/honda/wr-v/?offset=12">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/honda/wr-v/?offset=24">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/honda/wr-v/?offset=36">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/honda/wr-v/?offset=48">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/honda/wr-v/?offset=60">
<span aria-hidden="true"> </span>

In [112]:
df_wrv.to_csv('df_honda_wrv.csv', index=False, encoding='utf-8')
df_wrv

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,honda,wr-v,2022,NaN,Manual,16380000
1,honda,wr-v,2022,"71,000 km",Manual,14880000
2,honda,wr-v,2018,"76,022 km",Manual,10990000
3,honda,wr-v,2018,"69,000 km",Manual,10990000
4,honda,wr-v,2018,"49,500 km",Manual,11990000
...,...,...,...,...,...,...
77,honda,wr-v,2021,"19,000 km",Automática,15000000
78,honda,wr-v,2021,"12,300 km",Manual,15000000
79,honda,wr-v,2018,"67,000 km",Automática,12350000
80,honda,wr-v,2022,"6,800 km",Automática,17300000


In [114]:
df_mazda2 = contruir_df_auto('mazda', '2')

<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/mazda/2/?offset=12">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/mazda/2/?offset=24">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/mazda/2/?offset=36">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/mazda/2/?offset=48">
<span aria-hidden="true"> </span>
<span class="sr-only">Next</span>
</a>
<a aria-label="Next" class="page-link next" data-webm-clickvalue="sv-bpg-next" href="/vehiculos/autos-vehículo/mazda/2/?offset=60">
<span aria-hidden="true"> </span>
<span class="s

In [115]:
df_mazda2.to_csv('df_mazda2.csv', index=False, encoding='utf-8')
df_mazda2

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,mazda,2,2016,"75,000 km",Manual,11550000
1,mazda,2,2018,"72,900 km",Manual,10590000
2,mazda,2,2020,"62,000 km",Manual,11980000
3,mazda,2,2019,"26,000 km",Manual,12900000
4,mazda,2,2018,"36,000 km",Automática,12990000
...,...,...,...,...,...,...
139,mazda,2,2012,"133,000 km",Automática,7700000
140,mazda,2,2016,"106,000 km",Automática,10000000
141,mazda,2,2010,"113,519 km",Automática,7250000
142,mazda,2,2016,"83,300 km",Manual,10700000


In [21]:
df_mazda3 = contruir_df_auto('mazda', '3')

In [24]:
def cambiar_a_3(string):
    return 3

df_mazda3['Modelo'] = df_mazda3['Modelo'].apply(cambiar_a_3)

In [26]:
df_mazda3.to_csv('df_mazda3.csv', index=False, encoding='utf-8')
df_mazda3 = df_mazda3.dropna()
df_mazda3

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,mazda,3,2016,"72,000 km",Automática,13980000
1,mazda,3,2014,"87,300 km",Manual,11500000
2,mazda,3,2018,"88,940 km",Automática,13800000
8,mazda,3,2020,"31,000 km",Manual,15890000
9,mazda,3,2018,"66,000 km",Manual,14000000
...,...,...,...,...,...,...
150,mazda,3,2013,"155,000 km",Manual,7900000
151,mazda,3,2016,"68,000 km",Manual,15490000
152,mazda,3,2009,"170,000 km",Manual,5850000
153,mazda,3,2010,"169,000 km",Manual,6600000


In [121]:
df_mg3 = contruir_df_auto('mg', '3')

In [1]:
df_mg3.to_csv('df_mg3.csv', index=False, encoding='utf-8')
df_mg3['Modelo'] = df_mg3['Modelo'].apply(cambiar_a_3)  # aplicar esta función en otro archivo, abriendo el csv
df_mg3

NameError: name 'df_mg3' is not defined

In [128]:
df_kia_rio5 = contruir_df_auto('kia', 'rio-5')

In [130]:
df_kia_rio5.to_csv('df_kia_rio5.csv', index=False, encoding='utf-8')
df_kia_rio5

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,kia,2,2016,"130,000 km",Manual,9890000
1,kia,2,2017,"24,500 km",Manual,8690000
2,kia,2,2019,"51,179 km",Manual,10590000
3,kia,2,2019,"45,000 km",Manual,8930000
4,kia,2,2020,"54,000 km",Automática,12690000
...,...,...,...,...,...,...
139,kia,2,2011,"140,000 km",Manual,5980000
140,kia,2,2018,"62,983 km",Manual,10970000
141,kia,2,2018,"61,150 km",Manual,9450000
142,kia,2,2020,"49,882 km",Manual,11990000


In [10]:
df_crv = contruir_df_auto('honda', 'cr-v')

In [11]:
df_crv.to_csv('df_honda_crv.csv', index=False, encoding='utf-8')
df_crv

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,honda,cr-v,2017,"106,000 km",Automática,16790000
1,honda,cr-v,2015,"107,000 km",Automática,14500000
2,honda,cr-v,2017,"76,000 km",Automática,21580000
3,NaN,NaN,NaN,NaN,NaN,NaN
4,honda,cr-v,2023,NaN,Automática,33580000
...,...,...,...,...,...,...
143,honda,cr-v,2012,"93,000 km",Automática,12400000
144,honda,cr-v,2015,"122,685 km",Automática,13800000
145,honda,cr-v,2007,242 km,Automática,7000000
146,honda,cr-v,2014,"82,000 km",Automática,14590000


In [14]:
df_tiggo2 = contruir_df_auto('chery', 'tiggo-2')

In [15]:
df_tiggo2.to_csv('df_tiggo2.csv', index=False, encoding='utf-8')
df_tiggo2   #

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,chery,2,2021,"18,421 km",Manual,9980000
1,chery,2,2020,"61,203 km",Manual,8490000
2,chery,2,2018,"64,273 km",Manual,8290000
3,chery,2,2019,"54,100 km",Manual,7990000
4,chery,2,2020,"50,177 km",Manual,8490000
...,...,...,...,...,...,...
139,chery,2,2018,"70,530 km",Manual,9980000
140,chery,2,2021,"27,000 km",Automática,11480000
141,chery,2,2023,814 km,Manual,10490000
142,chery,2,2021,"16,000 km",Manual,9590000


In [17]:
df_baleno = contruir_df_auto('suzuki', 'baleno')

In [18]:
df_baleno.to_csv('df_baleno.csv', index=False, encoding='utf-8')
df_baleno

,Marca,Modelo,Año,Kilometraje,Transmision,Precio
0,suzuki,2,2021,"40,000 km",Manual,11950000
1,suzuki,2,2019,"55,000 km",Manual,7950000
2,suzuki,2,2022,"29,588 km",Manual,10990000
3,suzuki,2,2021,"51,622 km",Manual,10290000
4,suzuki,2,2020,"41,113 km",Manual,9490000
...,...,...,...,...,...,...
141,suzuki,2,2021,"7,000 km",Manual,11690000
142,suzuki,2,2018,"68,791 km",Manual,9680000
143,suzuki,2,2020,"61,650 km",Manual,10290000
144,suzuki,2,2022,"4,000 km",Manual,12200000
